In [ ]:
#| default_exp models.lstm

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# LSTM
The Long Short-Term Memory Recurrent Neural Network (`LSTM`), uses a multilayer `LSTM` encoder and an `MLP` decoder. It builds upon the LSTM-cell that improves the exploding and vanishing gradients of classic `RNN`'s. This network has been extensively used in sequential prediction tasks like language modeling, phonetic labeling, and forecasting.<br><br>**References**<br>-[Jeffrey L. Elman (1990). "Finding Structure in Time".](https://onlinelibrary.wiley.com/doi/abs/10.1207/s15516709cog1402_1)<br>-[Haşim Sak, Andrew Senior, Françoise Beaufays (2014). "Long Short-Term Memory Based Recurrent Neural Network Architectures for Large Vocabulary Speech Recognition."](https://arxiv.org/abs/1402.1128)<br>

![Figure 1. Long Short-Term Memory Cell.](imgs_models/lstm.png)

In [ ]:
#| hide
from nbdev.showdoc import show_doc

In [ ]:
#| export
import torch
import torch.nn as nn

from neuralforecast.losses.pytorch import MAE
from neuralforecast.common._base_recurrent import BaseRecurrent

In [ ]:
#| export
class LSTM(BaseRecurrent):
    def __init__(self,
                 input_size: int,
                 h: int,
                 state_hsize: int = 200, 
                 step_size: int = 1,
                 n_layers: int = 2,
                 activation: str = 'tanh',
                 bias: bool = True,
                 dropout: float = 0.,
                 learning_rate: float = 1e-3,
                 futr_exog_list = None,
                 hist_exog_list = None,
                 stat_exog_list = None,
                 loss=MAE(),
                 batch_size=32,
                 scaler_type: str='robust',
                 num_workers_loader=0,
                 drop_last_loader=False,
                 random_seed=1,
                 **trainer_kwargs):
        super(LSTM, self).__init__(
            h = h,
            input_size = input_size,
            loss=loss,
            learning_rate = learning_rate,
            batch_size=batch_size,
            scaler_type=scaler_type,
            futr_exog_list=futr_exog_list,
            hist_exog_list=hist_exog_list,
            stat_exog_list=stat_exog_list,
            num_workers_loader=num_workers_loader,
            drop_last_loader=drop_last_loader,
            random_seed=random_seed,
            **trainer_kwargs
        )

        # Architecture
        self.state_hsize = state_hsize
        self.step_size = step_size
        self.n_layers = n_layers
        self.activation = activation
        self.bias = bias
        self.dropout = dropout

        self.futr_exog_size = len(self.futr_exog_list)
        self.hist_exog_size = len(self.hist_exog_list)
        self.stat_exog_size = len(self.stat_exog_list)

        input_rnn = input_size + self.hist_exog_size*input_size + \
                    self.futr_exog_size*(input_size + h) + self.stat_exog_size

        # Instantiate model
        self.lstm = nn.LSTM(input_size=input_rnn,
                            hidden_size=self.state_hsize,
                            num_layers=self.n_layers,
                            bias=self.bias,
                            dropout=self.dropout,
                            batch_first=True)
        self.adapterW  = nn.Linear(self.state_hsize, self.h)

    def forward(self, windows_batch):
        
        # Parse windows_batch
        insample_y    = windows_batch['insample_y']
        futr_exog     = windows_batch['futr_exog']
        hist_exog     = windows_batch['hist_exog']
        stat_exog     = windows_batch['stat_exog']

        # Flatten inputs [B, W, C, L+H] -> [B, W, C*(L+H)]
        # Contatenate [ Y_t, | X_{t-L},..., X_{t} | F_{t-L},..., F_{t+H} | S ]
        batch_size, windows_size = insample_y.shape[:2]
        if self.hist_exog_size > 0:
            hist_exog = hist_exog.permute(0,2,1,3) # [B, C, W, L] -> [B, W, C, L]
            insample_y = torch.cat(( insample_y, hist_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        if self.futr_exog_size > 0:
            futr_exog = futr_exog.permute(0,2,1,3) # [B, C, W, L] -> [B, W, C, L]
            insample_y = torch.cat(( insample_y, futr_exog.reshape(batch_size, windows_size, -1) ), dim=2)

        if self.stat_exog_size > 0:
            stat_exog = stat_exog.unsqueeze(1).repeat(1, windows_size, 1) # [B, C] -> [B, W, C]
            insample_y = torch.cat(( insample_y, stat_exog ), dim=2)

        # LSTM forward
        insample_y, _ = self.lstm(insample_y)
        insample_y = self.adapterW(insample_y)
        
        return insample_y

In [ ]:
show_doc(LSTM)

In [ ]:
show_doc(LSTM.fit, name='LSTM.fit')

In [ ]:
show_doc(LSTM.predict, name='LSTM.predict')

## Usage Example

In [ ]:
#| eval: false
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from neuralforecast import NeuralForecast
from neuralforecast.losses.pytorch import MQLoss
from neuralforecast.utils import AirPassengersPanel, AirPassengersStatic
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader

Y_train_df = AirPassengersPanel[AirPassengersPanel.ds<AirPassengersPanel['ds'].values[-12]] # 132 train
Y_test_df = AirPassengersPanel[AirPassengersPanel.ds>=AirPassengersPanel['ds'].values[-12]].reset_index(drop=True) # 12 test

fcst = NeuralForecast(
    models=[LSTM(h=12,
                input_size=12,
                state_hsize=100,
                max_epochs=200,
                futr_exog_list=['trend'],
                hist_exog_list=['y_[lag12]'],
                stat_exog_list=['airline1']),
    ],
    freq='M'
)
fcst.fit(df=Y_train_df, static_df=AirPassengersStatic)
forecasts = fcst.predict(futr_df=Y_test_df)

# Plot
plot_df = Y_test_df.copy()
plot_df['LSTM'] = forecasts.reset_index(drop=False)['LSTM']
plot_df = pd.concat([Y_train_df, plot_df])

plot_df = plot_df[plot_df.unique_id=='Airline1'].drop('unique_id', axis=1)
plt.plot(plot_df['ds'], plot_df['y'], c='black', label='True')
plt.plot(plot_df['ds'], plot_df['LSTM'], c='blue', label='median')
plt.grid()
plt.legend()
plt.plot()